## Quick Start Agents with LangChain

[Tutorial](https://python.langchain.com/docs/modules/agents/quick_start/)

### Imports and Setup Keys

In [3]:
### Tracing
import os
import sys

sys.path.append('../../')  # replace with the actual path
import lib.key_param as key_param

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = key_param.LANGSMITH_API_KEY
os.environ['OPENAI_API_KEY'] = key_param.OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = key_param.TAVILY_API_KEY